In [1]:
import pandas as pd
import mysql.connector

def get_sql_type(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return 'INT'
    elif pd.api.types.is_float_dtype(dtype):
        return 'FLOAT'
    elif pd.api.types.is_bool_dtype(dtype):
        return 'BOOLEAN'
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return 'DATETIME'
    else:
        return 'TEXT'

In [2]:
# Define the file path and table name
file_path = r'C:\Another drive\Career\Data Scientist\Integrated_Projects\sql,excel,pyhon Coffee Shop Sales Analysis\CoffeeShop.csv'
table_name = 'coffee_shop_sales'

# MySQL connection details
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='06505',
    database='Coffee_shop_Sales_Analysis'  # Your specified database name
)

cursor = conn.cursor()


In [3]:
# Reading the CSV file into a pandas DataFrame
df = pd.read_csv(file_path)

# Replacing NaN with None to handle SQL NULL
df = df.where(pd.notnull(df), None)

# Cleaning column names
df.columns = [col.replace(' ', '_').replace('-', '_').replace('.', '_') for col in df.columns]


In [4]:
# Generating the create table statement
columns = ', '.join([f'`{col}` {get_sql_type(df[col].dtype)}' for col in df.columns])
create_table_query = f'CREATE TABLE IF NOT EXISTS `{table_name}` ({columns})'
cursor.execute(create_table_query)


In [5]:
# Inserting DataFrame data into the MySQL table
for _, row in df.iterrows():
    # Convert row to tuple and handle NaN/None explicitly
    values = tuple(None if pd.isna(x) else x for x in row)
    sql = f"INSERT INTO `{table_name}` ({', '.join(['`' + col + '`' for col in df.columns])}) VALUES ({', '.join(['%s'] * len(row))})"
    cursor.execute(sql, values)

# Commit the transaction
conn.commit()
# Close the connection
cursor.close()
conn.close()
